In [4]:
import pandas as pd
import jsonlines
import numpy as np

pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_colwidth', 500)  # Display full text in columns


## Datasets

In [5]:
# Analyzing Political Bias and Unfairness in News Articles at Different Levels of Granularity
# Political Ideology and Polarization: A Multi-dimensional Approach

import json

with open("data/released_data.json", "r") as file:
    j_string = file.readline()
    jsons = []
    while j_string:
        data = json.loads(j_string)
        jsons.append(data)
        j_string = file.readline()
        
df = pd.DataFrame(jsons)

df_right_bias = df.loc[(df['allsides_bias'] == 'From the Right') & (df['adfontes_political'] == 'bias') & (df['adfontes_fair'] == 'bias')] 

df_left_bias = df.loc[(df['allsides_bias'] == 'From the Left') & (df['adfontes_political'] == 'bias') & (df['adfontes_fair'] == 'bias')] 

df.tail(6)


,source,title,event_id,adfontes_fair,adfontes_political,allsides_bias,content,misc
7769,Washington Times,PICKET: RNC's Priebus hits back at MSNBC's Matthews - 'He wants to be the biggest jerk in the room',0,bias,neutral,From the Right,"*UPDATE 8/29/12 - MSNBC’s Chris Matthews Responded to Chairman Priebus:\n“Nobody’s asked me to apologize,” said MSNBC’s Chris Matthews. “I was telling the truth…those ads are aimed at racial purpose,” he later added.","{'time': 'None', 'topics': 'Election: Presidential', 'author': ['Kerry Picket']}"
7770,Politico,Ann Romney's task: Humanize Mitt,0,bias,neutral,From the Left,"TAMPA, Fla. — Ann Romney will take to the podium here Tuesday night and try to accomplish what the sharpest minds in Republican politics have failed to do: present her stiff and awkward husband as a likable guy.\nHer speech is considered second in importance only to Mitt Romney’s and will set the tone for the week as the Republican Party presents its case for turning President Barack Obama out of office.\nStory Continued Below\nConvention and campaign officials see Ann Romney as the only per...","{'time': '2012-08-28 04:59:14', 'topics': 'Election: Presidential', 'author': ['LOIS ROMANO', 'Frida Ghitis', 'John A. Farrell', 'Jeff Greenfield', 'Cliff Sims']}"
7771,Washington Times,'Mittigator' to make case for Romney,0,bias,neutral,From the Right,"She is a gracious warrior with a kind face, a polished demeanor and the inner mettle of a survivor. Ann Romney has been called the secret weapon of her husband’s campaign, and he adores her: When Mitt Romney strides to the podium to introduce his wife of 43 years, he calls her “sweetheart.” She smiles. The crowd goes crazy.\nAnd why shouldn’t they? Though the Romneys could emerge as the nation’s No. 1 power couple, there is a certain authenticity about them, built over decades around family ...","{'time': 'None', 'topics': 'Election: Presidential', 'author': ['Jennifer Harper']}"
7772,Fox News,"Convention-bound Ryan slams Obama for presiding over 'debt, doubt and decline'",0,bias,bias,From the Right,"Republican VP pick on 'Special Report'\nMaking one last stop in his hometown before flying to Tampa for the official nomination to the Republican ticket, vice presidential pick Paul Ryan slammed the current White House for putting the nation down a road of ""debt, doubt and decline.""\nIn an interview with Fox News&apos Bret Baier, Ryan teed up his speech at the Republican National Convention by stressing that the weather-delayed event will nevertheless provide a critical opportunity to draw a...","{'time': '2012-08-27 00:00:00', 'topics': 'Election: Presidential', 'author': ['| Fox News']}"
7773,Politico,Ryan seeks comfort of Ron Paul fans,0,bias,neutral,From the Left,"Paul Ryan said Monday he expects Ron Paul supporters will come around to the Republican ticket and “should be very comfortable with us.”\nRyan told Fox News that he and Paul “see eye to eye on a lot of issues,” adding that he doesn’t feel there is any tension with Paul — who held his own rally in Tampa on Sunday and did not endorse Romney — or the Texas congressman’s supporters.\nStory Continued Below\n“So I think, in the final analysis, Ron is clearly going to — he and his supporters should...","{'time': '2012-08-27 15:52:45', 'topics': 'Election: Presidential', 'author': ['MACKENZIE WEINGER', 'Frida Ghitis', 'John A. Farrell', 'Jeff Greenfield', 'Cliff Sims']}"
7774,CNN (Web News),Paul to supporters: 2012 the start of a new era,0,bias,neutral,From the Left,"Tampa, Florida (CNN) \- As Rep. Ron Paul's third presidential bid is set to come to its formal end at the Republican National Convention, Paul sought to rally his loyal supporters with talk of a new era in American history.\n""I'm convinced that we're living at the end of an era,"" Paul told the crowd at a rally in honor of him and his libertarian political movement. ""The end of an era provides an opportunity,"" Paul added before laying out his view that an era in A

In [6]:
df_right_bias.sample(3)

,source,title,event_id,adfontes_fair,adfontes_political,allsides_bias,content,misc
4804,Fox News,Security Expert: My Fear is That More Churches Will be Targeted,0,bias,bias,From the Right,"South Carolina governor delivers emotional statement after Charleston church massacre\nThe gunman walked into the church sanctuary and opened fire.\nHe showed no mercy – firing more than 100 rounds and exploding a pipe bomb in the balcony. There was nowhere to hide from the carnage.\nClick here to follow Todd on Facebook for conservative conversation!\nSeven people died that night. Seven others were wounded.\nIt was Sept. 15, 1999 – the day a madman slaughtered Christians inside Wedgwood Bap...","{'time': '2015-06-18 00:00:00', 'topics': 'Violence in America', 'author': ['Todd Starnes', '| Fox News']}"
4321,Fox News,Turnout is name of the game in Iowa caucuses,0,bias,bias,From the Right,"The 2016 presidential contenders are begging their Iowa supporters to get to the caucuses Monday and Donald Trump, true to form, is in-your-face about it.\n""You&aposre from Iowa,"" Trump told a Dubuque crowd Saturday. ""Are you afraid of snow?""\nA snowfall forecast to start Monday night appeared more likely to hinder the hopefuls in their rush out of Iowa than the voters who will be flocking to the caucuses in the first contest of the presidential campaign.\nStill, there was every reason for c...","{'time': '2016-01-31 00:00:00', 'topics': 'Election: Presidential', 'author': ['| Associated Press']}"
4037,Newsmax,Ted Cruz Desperately Seeks to Stop Donald Trump in Indiana,0,bias,bias,From the Right,"Republican Ted Cruz faces a high-stakes test for his slumping presidential campaign in Tuesday's Indiana primary, one of the last opportunities for the Texas senator to halt Donald Trump's stunning march toward the GOP nomination. Cruz has spent the past week camped out in Indiana, securing the support of the state's governor and announcing retired technology executive Carly Fiorina as his running mate. Yet his aides were pessimistic heading into Tuesday's voting and were prepared for Cruz t...","{'time': '2016-05-03 06:42:00', 'topics': 'Election: Presidential', 'author': ['JULIE PACE and SCOTT BAUER']}"


In [7]:
csv_file = "data/toxicbias_train.csv"
df = pd.read_csv(csv_file)

# Display the DataFrame
df.sample(5)


,comment_text,identity_attack_x,bias,target,category,rationale
2842,Also prohomophobic protraitorstoAmerica proNO HEALTHCARE promisogyny profascist dictators ...,0.2875,neutral,NaN,none,NaN
3142,If we should blame all Muslims for the acts of a few as many on the right feel we should be doing then yes we should blame all Christians for the actions of this one guy.,0.8000,neutral,NaN,none,NaN
4234,Christianity is the Religion of Love except for the part where Catholic and Protestant want to kill each other riight.,0.5125,bias,christians,religion,prejudice towards christians
4190,hipsters white people with dreadlocks credit card hippies and selfentitled selfrighteous over privileged idealistic white millenials. if i were a minority a woman or gay these would be the absolute last people i would want taking up my cause or attempting to represent my interests andor frustrations. but alas i am in fact the problem a thirtysomething white male the sole cause of every injustice ever perpetrated on anyone or anything since the damn of humankind. and ive never been so ash...,0.8000,bias,whites,race,prejudice against whites
2602,Whites commit the majority of crime in the U.S. What do you think we should do about it?,0.5875,bias,whites,race,prejudice against whites


## Model

In [36]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

def predict_nli_label(premise, hypothesis):
    model = AutoModelForSequenceClassification.from_pretrained('cross-encoder/nli-deberta-base')
    tokenizer = AutoTokenizer.from_pretrained('cross-encoder/nli-deberta-base')

    features = tokenizer([premise], [hypothesis],  padding=True, truncation=True, return_tensors="pt")

    model.eval()
    with torch.no_grad():
        scores = model(**features).logits
        label_mapping = ['contradiction', 'entailment', 'neutral']
        labels = [label_mapping[score_max] for score_max in scores.argmax(dim=1)]

    return labels[0], scores.tolist()[0] 


In [9]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model='cross-encoder/nli-deberta-base')

sent = "Hey he hates Asians too Apparently successful minorities get under the loony lefts skin."
candidate_labels = ["Prejudice", "Wealth", "Income"]
res = classifier(sent, candidate_labels)
print(res)

{'sequence': 'Hey he hates Asians too Apparently successful minorities get under the loony lefts skin.', 'labels': ['Wealth', 'Prejudice', 'Income'], 'scores': [0.382228285074234, 0.3719187080860138, 0.245852991938591]}


## Test examples

In [37]:
premise = "Hey he hates Asians too Apparently successful minorities get under the loony lefts skin."
hypothesis = "Prejudice towards individuals based on geographic location."

predict_nli_label(premise,hypothesis)

('neutral', [-1.519162893295288, -0.20995476841926575, 1.3938082456588745])

In [34]:
premise = "Intolerant liberals here calling others intolerant. Liberalism is a mental disorder. The sooner we rub it out of Western culture the better."
hypothesis = "Dislike, hatred, or prejudice against a political group, in particular liberals."

predict_nli_label(premise,hypothesis)

('entailment', [-3.1031978130340576, 1.9444491863250732, 0.7191958427429199])

In [35]:
premise = "Its bad enough we already have all these American sickos living in this country shooting up theatre schools churches malls etc. we dont have room for crazy muslims."
hypothesis = "Dislike, hatred or prejudice towards individuals of a race."

predict_nli_label(premise,hypothesis)

('entailment', [-1.6237410306930542, 1.1280356645584106, 0.0267395731061697])

In [33]:
premise = "where is our vocal feral cat killing muslim to defend his fellow islamist? must be hiding under his blow up sheep."
hypothesis = "Dislike, hatred, or prejudice towards individuals who are muslim."

predict_nli_label(premise,hypothesis)

('entailment', [-1.1952913999557495, 0.5450257062911987, 0.3316133916378021])

In [32]:
premise = "where is our vocal feral cat killing muslim to defend his fellow islamist? must be hiding under his blow up sheep."
hypothesis = "Love or positive feelings towards individuals who are muslim."
predict_nli_label(premise,hypothesis)

('contradiction',
 [1.3223092555999756, -0.9107821583747864, -0.4003891348838806])

In [15]:
premise1 = "Islam frightens me when I think of my daughters future in an islamized Canada."
premise2= "This is not an irrational fear not a phobia when you look at whats happening in the world today."
hypothesis = "Dislike, hatred or prejudice towards individuals who are muslim."
hypothesis_New = "Negative feelings towards islam."

predict_nli_label(premise1,hypothesis_New)
predict_nli_label(premise2,hypothesis_New)

tensor([[-1.5974,  1.0171,  0.1459]])
['entailment']
tensor([[ 0.4719, -2.0669,  1.2745]])
['neutral']


In [16]:
premise = "Islam frightens me when I think of my daughters future in an islamized Canada. This is not an irrational fear not a phobia when you look at whats happening in the world today."
hypothesis = "Negative view of individuals who are muslim."

predict_nli_label(premise,hypothesis)

tensor([[ 1.1562, -1.3025,  0.0457]])
['contradiction']


In [17]:
premise = "Islam frightens me when I think of my daughters future in an islamized Canada. This is not an irrational fear not a phobia when you look at whats happening in the world today."
hypothesis = "Fear of individuals who are protestant."

predict_nli_label(premise,hypothesis)

tensor([[-0.0936, -0.1933,  0.1388]])
['neutral']


In [41]:
premise = "Great. Catholics are on the winning side and the country as a whole loses. Whats not to love? Make America hate again."
hypothesis = "Derogatory negative opinion against individuals"

predict_nli_label(premise,hypothesis)

('entailment', [-1.183259129524231, 0.4223703444004059, 0.39567989110946655])

## Initial testing 

Using exisiting rationale from dataset to see if model gives similar results

In [40]:
csv_file = "data/toxicbias_train.csv"
df = pd.read_csv(csv_file)

# Create empty lists to store the predicted labels and scores
predicted_labels = []
predicted_scores = []

# Iterate over each row in the dataset
for index, row in df.iterrows():
    comment_text = row['comment_text']
    rationale = row['rationale']

    # Check if rationale is NaN
    if pd.isna(rationale):
        rationale = "No bias or prejudice in the text."
        
    label, scores = predict_nli_label(comment_text, rationale)
    
    predicted_labels.append(label)
    predicted_scores.append(scores)

# Add the predicted labels and scores to the dataset
df['predicted_label'] = predicted_labels
df['predicted_scores'] = predicted_scores


In [44]:
# save result to csv
df.to_csv('results/output.csv', index=False)

In [45]:
df.sample(10)

,comment_text,identity_attack_x,bias,target,category,rationale,predicted_label,predicted_scores
2863,Yet these same groups look the other way when islam teaches it is O.K. to beat women to rape women for not dressing properly and allows the raping of non Islamic women The hypocrisy is staggering,0.571429,bias,muslims,religion,prejudice against muslims,neutral,"[-1.7085249423980713, -0.16685394942760468, 1.5042953491210938]"
576,They should all be given long sentences and thrown in prison like they did during the crack epidemic but of course that situation involved a bunch o black and brown people so now its treatment sympathy and a public health emergency. You got to love America...that great Christian Nation. Unbelievable,0.500000,bias,"blacks,christians","race,religion",prejudice against blacks and christians,contradiction,"[0.5061009526252747, -1.0798803567886353, 0.41280320286750793]"
1709,Thats disturbing that an atheist and presumably an agnostic could never be voted into office. And its also disturbing that religion or any of these categories automatically would have an affect on votes.,0.500000,bias,atheists,religion,prejudice against atheists,neutral,"[-1.8708322048187256, -0.6543827652931213, 2.1548099517822266]"
4039,these people were hired to work for the AMERICAN people not refugee muslims VOTE THEM OUT,0.800000,bias,muslims,religion,prejudice towards muslims,neutral,"[-0.21673975884914398, -1.2978944778442383, 1.109818935394287]"
729,Theres little difference between Muslim terrorists and Israeli terrorists.,0.800000,bias,"muslims,israelis","religion,race","prejudice against muslims,israelis",contradiction,"[2.2116761207580566, -1.6123117208480835, -0.6242567896842957]"
2764,Bruddah Trump just got back from the Middle East where he was welcomed with open arms yet people like you and the media still trying to peg him as an antiMuslim racist. No can,0.257143,neutral,NaN,none,NaN,contradiction,"[2.1651017665863037, -1.634959101676941, -0.541456937789917]"
33,Being antiracist isnt racist. The right has tried to peddle that antiwhite nonsense for two generations now since the GOP adopted all the orphaned racist southerners abandoned by the Democrats. No one has said border control is racist. Youre engaging in whats known as equivocation.,0.500000,bias,whites,race,prejudice against whites,neutral,"[-0.39023658633232117, -0.5655011534690857, 0.6482449173927307]"
1573,Perhaps the women were black as well as Muslim. From the article: Portland police Sgt. Pete Simpson says the assailant on the train was ranting on many topics using hate speech or biased language and turned his focus on the women.,0.500000,bias,"blacks,muslims","race,religion",prejudice against blacks and muslims,entailment,"[-3.069397211074829, 1.6497869491577148, 0.8996347784996033]"
1314,Wah wah wah go paint yourself a sign and walk around in a circle with the other morons. I just got back from a gay wedding and smoked a huge bowl of the weed I grew. You are a narrow minded lefty who hasnt got a clue.,0.274194,neutral,NaN,none,NaN,contradiction,"[2.466484546661377, -2.1830663681030273, -0.3328106701374054]"
4129,go peddle your white mans guilt elsewhere.,0.525000,bias,"whites,men","race,gender",prejudice towards white men,entailment,"[-1.1454640626907349, 0.5496312975883484, 0.2691248953342438]"


In [53]:
label_counts = df['predicted_label'].value_counts()

entailment_count = label_counts.get('entailment', 0)
contradiction_count = label_counts.get('contradiction', 0)
neutral_count = label_counts.get('neutral', 0)

print(f'Total: {entailment_count + contradiction_count + neutral_count}')
print(f'Entailment total: {entailment_count}')
print(f'Contradiction total: {contradiction_count}')
print(f'Neutral total: {neutral_count}')


Total: 4327
Entailment total: 507
Contradiction total: 1025
Neutral total: 2795
